In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.preprocessing import MinMaxScaler
import pickle
import torch
from models import *

In [2]:
def data_processing(data):
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    categorical_features = list(data.select_dtypes(include='object').columns)
    categorical_features = list(set(categorical_features))
    numerical_features = [c for c in data.columns if c not in categorical_features]
    scaler = MinMaxScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])

    for _c in categorical_features:
        data[_c] = pd.Categorical(data[_c])
    df_transformed = pd.get_dummies(data, drop_first=False)
    return df_transformed, scaler


In [3]:
import numpy as np
dataset = 'music_data.csv'
dataset = pd.read_csv(dataset).iloc[:, 1:]
dataset, scaler = data_processing(dataset)
y = np.array(dataset.iloc[:, -5:], dtype=np.float32)
X = np.array(dataset.iloc[:, :-5], dtype=np.float32)


In [4]:
from sklearn.metrics import pairwise_distances
def infy_pairwise_distance(a, b):
    return max(abs(a - b))

pairwise_distances(X, X, metric=infy_pairwise_distance).mean()

0.8908711296178018

In [8]:
folders = range(1, 6)
baseline = 'sl'
processed_baseline_results = []
for f in folders:
    results_baseline = pd.read_csv('{}/baseline_output_{}.csv'.format(f, baseline)).iloc[:, 1:]
    for i in range(results_baseline.shape[0]):
        processed_baseline_results.append([f, i + 1, results_baseline.iloc[i, :]['accuracy test corrected'], 
                                           results_baseline.iloc[i, :]['violations test'],
                                          results_baseline.iloc[i, :]['runtime']])

processed_baseline_results = pd.DataFrame(processed_baseline_results)
processed_baseline_results.columns = ['exp num', 'fold', 'test accuracy', 'test violations', 'runtime']
processed_baseline_results

,exp num,fold,test accuracy,test violations,runtime
0,1,1,0.797386,0.0,105.055259
1,1,2,0.842105,0.0,124.774570
2,1,3,0.878378,0.0,86.032901
3,1,4,0.792208,0.0,87.026860
4,1,5,0.856164,0.0,179.018016
5,2,1,0.826667,0.0,53.074240
6,2,2,0.766667,0.0,27.550068
7,2,3,0.828947,0.0,101.717340
8,2,4,0.816993,0.0,117.737622
9,2,5,0.898649,0.0,175.931137


In [9]:
print(np.mean(processed_baseline_results['test accuracy']), 
      np.std(processed_baseline_results['test accuracy']),
      np.mean(processed_baseline_results['test violations']), 
      np.std(processed_baseline_results['test violations']),
     np.mean(processed_baseline_results['runtime']), 
      np.std(processed_baseline_results['runtime']))

0.8253547590646374 0.03580591763901195 0.16 0.6118823416311343 101.06969027519226 45.848151252562666


for each experiment, we read the models for each fold and aclculate adversarial robustness

In [9]:
def predict(model, X):
    X_torch = np.array(X)
    X_torch = X_torch.astype(np.float32)
    X_torch = torch.tensor(X_torch)
    
    score = model.forward(X_torch).detach().numpy()
    pred = []
    for p in score:
        pred.append(np.argmax(p))
    pred_d = np.zeros((len(pred), len(score[0])))
    pred_d[np.arange(len(pred)), pred] = 1
    return pred_d

from sklearn.metrics import accuracy_score
def calculate_corrected_accuracy(X, pred, y, K, corrected=True):
    if corrected:
        scaled_income = K.scale_[0] * (5000 - K.data_min_[0])
        indices_to_remove = np.where((X.T[0] >= 0) & (X.T[0] <= scaled_income) & 
                                     (X.T[4] == 0) & (y.T[1] == 1))[0]
        indices = list(set(range(len(X))) - set(indices_to_remove))
        return accuracy_score(pred[indices], y[indices])
    else:
        return accuracy_score(pred, y)

In [10]:
def get_adi(model, X, delta = 0.1):
    torch.onnx.export(model,                 # model being run
                  torch.tensor(X[:5]),       # model input (or a tuple for multiple inputs)
                  "baseline.onnx",           # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    onnx_model = onnx.load("baseline.onnx")
    onnx.checker.check_model(onnx_model)
    
    adi = 0
    for instance in X:
        network = Marabou.read_onnx(filename="baseline.onnx")
        inputVars = network.inputVars[0][0]
        outputVars = network.outputVars[0][0]
        ranges = [[0, 1]]*len(inputVars)
        ranges[12] = [1, 1]
        instance[12] = 1
                    
        for i in range(len(instance)):
            network.setLowerBound(inputVars[i], max(ranges[i][0], instance[i] - delta))
            network.setUpperBound(inputVars[i], min(ranges[i][1], instance[i] + delta))
        
        network.addInequality([16], [1], -0.001)
        network.addInequality([17], [1], -0.001)
        
        eq1 = MarabouCore.Equation(MarabouCore.Equation.GE)
        eq1.addAddend(1, 13)
        eq1.setScalar(0.001)
        
        eq2 = MarabouCore.Equation(MarabouCore.Equation.GE)
        eq2.addAddend(1, 14)
        eq2.setScalar(0.001)

        eq3 = MarabouCore.Equation(MarabouCore.Equation.GE)
        eq3.addAddend(1, 15)
        eq3.setScalar(0.001)
        
        network.addDisjunctionConstraint([[eq1], [eq2], [eq3]])
        
        options = Marabou.createOptions(verbosity = 2)
        vals = network.solve(options = options)
#         sys.exit(0)

        if vals[0] == 'sat':
            adi += 1
    return adi/len(X)


In [ ]:
import onnx
from maraboupy import Marabou, MarabouCore

for e in folders:
    output = []
    for f in range(0, 5):
        model = torch.load(str(e) + '/baseline_model_fold_{}_{}.pt'.format(f + 1, baseline))
        pred = model.predict(X)
        for delta in [0.1, 0.01]:
            adi = get_adi(model, X, delta=delta)
            output.append([e, f + 1, delta, adi])
    output = pd.DataFrame(output)
    output.columns = ['experiment', 'fold', 'delta', 'adi']
    output.to_csv('{}/adi_{}.csv'.format(e, baseline))


Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


unsat
unsat
unsat
unsat
unsat
unsat
unsat
sat
input 0 = 1.0
input 1 = 0.22754519581794738
input 2 = 0.3333296906357595
input 3 = 0.8458738565444947
input 4 = 0.5437787312905081
input 5 = 0.4584137395799464
input 6 = 0.1
input 7 = 0.015649966884194143
input 8 = 0.0
input 9 = 0.07650563272428308
input 10 = 0.0
input 11 = 0.0
input 12 = 1.0
output 0 = 0.001
output 1 = -2.421961481161506
output 2 = -5.487635125871722
output 3 = -1.1099055664115332
output 4 = -3.635340568846636
unsat
unsat
unsat
unsat
sat
input 0 = 0.9500279074619599
input 1 = 0.7322142481803894
input 2 = 0.4748885691165924
input 3 = 0.7113744974136352
input 4 = 0.5661688208580017
input 5 = 0.6717927098274231
input 6 = 0.05861122259053426
input 7 = 0.0
input 8 = 0.0
input 9 = 0.0
input 10 = 0.1
input 11 = 0.1
input 12 = 1.0
output 0 = -8.218764524979182
output 1 = -1.6451062220816839
output 2 = 0.001
output 3 = -1.1034101184350529
output 4 = -3.602003187774927
sat
input 0 = 0.8758165001869201
input 1 = 0.4013287114562193
in

In [13]:
adi_results = pd.DataFrame()
for e in folders:
    adi = pd.read_csv(str(e) + '/adi_{}.csv'.format(baseline)).iloc[:, 1:]
    adi_results = adi_results.append(adi)

In [14]:
for delta in [0.1, 0.01]:
    print("adi for delta = {} is {} +/- {}".format(delta, 
                                                   np.mean(adi_results[adi_results.delta == delta].adi.tolist()), 
                                                   np.std(adi_results[adi_results.delta == delta].adi.tolist())))
    

adi for delta = 0.1 is 0.2931147540983607 +/- 0.10223561949016106
adi for delta = 0.01 is 0.21684741488020173 +/- 0.08705372435267582


In [10]:
# folders = range(1, 8)
processed_deepsade_results = []
for f in folders:
    results_deepsade = pd.read_csv('{}/deepsade_output_ls.csv'.format(f)).iloc[:, 1:]
    mse_mean = np.mean(results_deepsade['accuracy test corrected'])
    violations_mean = np.mean(results_deepsade['violations test'])
    for i in range(results_deepsade.shape[0]):
        processed_deepsade_results.append([f, i + 1, results_deepsade.iloc[i, :]['accuracy test corrected'], 
                                           results_deepsade.iloc[i, :]['violations test'],
                                          results_deepsade.iloc[i, :]['runtime']])

processed_deepsade_results = pd.DataFrame(processed_deepsade_results)
processed_deepsade_results.columns = ['exp num', 'fold', 'test accuracy', 'test violations', 'runtime']
processed_deepsade_results

,exp num,fold,test accuracy,test violations,runtime
0,1,1,0.790850,0.0,8668.390595
1,1,2,0.842105,0.0,7057.123867
2,1,3,0.871622,0.0,4453.496365
3,1,4,0.779221,0.0,5690.095434
4,1,5,0.821918,0.0,5713.619134
5,2,1,0.733333,0.0,5303.681205
6,2,2,0.686667,0.0,4882.271661
7,2,3,0.809211,0.0,5228.449211
8,2,4,0.790850,0.0,6497.902207
9,2,5,0.891892,0.0,4942.538579


In [11]:
print(np.mean(processed_deepsade_results['test accuracy']), 
      np.std(processed_deepsade_results['test accuracy']),
      np.mean(processed_deepsade_results['test violations']), 
      np.std(processed_deepsade_results['test violations']),
     np.mean(processed_deepsade_results['runtime']), 
      np.std(processed_deepsade_results['runtime']))

0.8011324212515486 0.049952075972994024 0.0 0.0 6580.688732490539 1915.2163454952195
